In [ ]:
! pip install pyradiomics
! pip install torch-summary

In [ ]:
# importing libararies
import cv2
import os
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchsummary import summary

import nibabel as nib
import scipy.ndimage as ndi
from sklearn.preprocessing import LabelBinarizer

import radiomics
from radiomics import featureextractor
import SimpleITK as sitk
import six
from sklearn.preprocessing import StandardScaler

## Paths

In [ ]:
# train data
directory_path_train = '/kaggle/input/brats2018/MICCAI_BraTS_2018_Data_Training'
high_grade_glioma_train = '/kaggle/input/brats2018/MICCAI_BraTS_2018_Data_Training/HGG'
low_grade_glioma_train = '/kaggle/input/brats2018/MICCAI_BraTS_2018_Data_Training/LGG'
survival_csv_train = '/kaggle/input/brats2018/MICCAI_BraTS_2018_Data_Training/survival_data.csv'
# val data
directory_path_val = '/kaggle/input/brats2018/MICCAI_BraTS_2018_Data_Validation'
survival_csv_val = '/kaggle/input/brats2018/MICCAI_BraTS_2018_Data_Validation/survival_evaluation.csv'

## Handling the csv file

### Encoding scheme of resection status
#### NaN : [0, 0]
#### STR : [0, 1]
#### GTR : [1, 0]

In [ ]:
resection_status = { 'nan' : [0, 0], 'STR': [0, 1], 'GTR': [1, 0]} 

In [ ]:
# reading the data from CSV file
train_survival = pd.read_csv(survival_csv_train)
train_survival_dict = train_survival.set_index('BraTS18ID').T.to_dict('list')

## Function for feautre vector of a segmented MRI

In [ ]:
def vol_and_SA(extractor, image, mask):
    feautres = extractor.execute(image, mask) 
    MeshVolume = 0
    SurfaceArea = 0
    for key, value in feautres.items():
        if key == 'original_shape_MeshVolume':
            MeshVolume = value
        elif key == 'original_shape_SurfaceArea':
            SurfaceArea = value
            
    return MeshVolume, SurfaceArea

In [ ]:
def get_mask(seg_mri, masked_val):
    mask = np.zeros_like(seg_mri)
    for slice in range(0, seg_mri.shape[0]):
        for row in range(0, seg_mri.shape[1]):
            for col in range(0, seg_mri.shape[2]):
                if seg_mri[slice][row][col] == masked_val:
                    mask[slice][row][col] = 1
    
    return mask

In [ ]:
def get_feautre_vector(seg_mri):
    feautre_vector = []
    
    necrotic_net_mask = get_mask(seg_mri, masked_val = 1) # 0 - background, 1 - necrotic and non_enhancing tumor
    peritumoral_edema_mask = get_mask(seg_mri, masked_val = 2) # 0 - background, 1 - peritumoral edema
    enhancing_tumor_mask = get_mask(seg_mri, masked_val = 4) # 0 - background, 1 - enhancing tumor
    
    seg_image = sitk.GetImageFromArray(seg_mri)
    necrotic_net_mask_image = sitk.GetImageFromArray(necrotic_net_mask)
    peritumoral_edema_mask_image = sitk.GetImageFromArray(peritumoral_edema_mask)
    enhancing_tumor_mask_image = sitk.GetImageFromArray(enhancing_tumor_mask)
    
    extractor = featureextractor.RadiomicsFeatureExtractor()
    MeshVolume_necrotic_net, SurfaceArea_necrotic_net = vol_and_SA(extractor, seg_image, necrotic_net_mask_image)
    MeshVolume_peritumoral_edema, SurfaceArea_peritumoral_edema = vol_and_SA(extractor, seg_image, peritumoral_edema_mask_image)
    MeshVolume_enhancing_tumor, SurfaceArea_enhancing_tumor = vol_and_SA(extractor, seg_image, enhancing_tumor_mask_image)
#     print(type(MeshVolume_necrotic_net))
    feautre_vector.append(float(MeshVolume_necrotic_net))
    feautre_vector.append(float(SurfaceArea_necrotic_net))
    feautre_vector.append(float(MeshVolume_peritumoral_edema))
    feautre_vector.append(float(SurfaceArea_peritumoral_edema))
    feautre_vector.append(float(MeshVolume_enhancing_tumor))
    feautre_vector.append(float(SurfaceArea_enhancing_tumor))
    
    return feautre_vector

## Get feautre vector for regressor model

In [ ]:
def get_input_GT_data(dir_path):
    input_data = []
    gt = []
    i = 0
    main_files = os.listdir(dir_path)
    for main_file in main_files:
        full_main_path = os.path.join(dir_path, main_file)
        try : 
            files = os.listdir(full_main_path)
        except NotADirectoryError : 
            continue
        for file in files:
            full_dir_path = os.path.join(full_main_path, file)
            vimage_files = os.listdir(full_dir_path)
#             print(file)
            try : 
                age, survival, resection = train_survival_dict[file]
                resection_str = str(resection)
            except KeyError :
                continue
            for vimage_file in vimage_files:
                if "seg" in vimage_file:
                    vimage_path = os.path.join(full_dir_path, vimage_file)
                    print(vimage_file)                      
                    brain_vol = nib.load(vimage_path)
                    brain_vol_data = brain_vol.get_fdata()
                    feautre_vector = get_feautre_vector(brain_vol_data)
                    feautre_vector.append(age)
                    print(resection_str)
                    feautre_vector.append(resection_status[resection_str][0])
                    feautre_vector.append(resection_status[resection_str][1])
                    feautre_vector.append(file)
                    feautre_vector.append(survival)
                    input_data.append(feautre_vector)
                    gt.append(survival)
            if i == 20:
                break
            i = i + 1
    return (input_data, gt)

# reading the MRI train images
(input_data, gt) = get_input_GT_data(directory_path_train)

### Saving the feautres to csv file 

In [ ]:
pd.DataFrame(input_data).to_csv('data_input_and_labels3.csv')

In [ ]:
# Reshaping
input_d = []
for i in range(21):
    input_d.append(input_data[i][0 : -2])

### Visualise a MRI

In [ ]:
# visualise each slice
def visualise_each_slice(brain_vol_data):
    fig_rows = 4
    fig_cols = 4
    n_subplots = fig_rows * fig_cols
    n_slice = brain_vol_data.shape[0]
    step_size = n_slice // n_subplots
    plot_range = n_subplots * step_size
    start_stop = int((n_slice - plot_range) / 2)
    fig, axs = plt.subplots(fig_rows, fig_cols, figsize = [20, 20])

    for idx, img in enumerate(range(start_stop, plot_range, step_size)):
        axs.flat[idx].imshow(ndi.rotate(brain_vol_data[img, :, :], 90))
        print(img)
        axs.flat[idx].axis('off')

    plt.tight_layout()
    plt.show()

## Dataset for Linear regressor, KNN and decision tree

In [ ]:
# data 
X = input_d
y = gt

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

X_train_copy = X_train.copy()
X_test_copy = X_test.copy()
 
# Fit the standardization scaler onto the training data
standard = StandardScaler().fit(X_train_copy)
 
# Transform the training data
X_train_stan = standard.transform(X_train_copy)
X_test_stan = standard.transform(X_test_copy)

X_train_tensor = torch.tensor(X_train_stan, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_stan, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## Regressor Model

In [ ]:
# Define the model and loss functions
class LinearRegressor(nn.Module):
  def __init__(self):
    super(LinearRegressor, self).__init__()
    self.fc1 = nn.Linear(9, 1)  # Input layer with 9 features and 1 output

  def forward(self, x):
    x = self.fc1(x)
    return x

mse_loss = nn.MSELoss()

def median_squared_error(y_true, y_pred):
  squared_errors = F.mse_loss(y_true, y_pred, reduction = 'none')
  median_error = torch.median(squared_errors)
  return median_error

def combined_loss(y_true, y_pred, weight_mse = 0.5, weight_median = 0.5):
  mse = mse_loss(y_true, y_pred)
  median_error = median_squared_error(y_true, y_pred)
  return weight_mse * mse + weight_median * median_error

In [ ]:
# Training loop
def train(model, optimizer, data_loader, target_loader, epochs = 10):
    losses = []
    for epoch in range(epochs):
        for data, target in zip(data_loader, target_loader):  # Iterate through batches in the dataloader
            optimizer.zero_grad()  # Clear gradients before each pass
#             print(data, target)
            y_pred = model(data)  # Forward pass
            loss = combined_loss(target, y_pred)  # Calculate combined loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model weights

#       Print training losses
        losses.append(loss.item())
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    plt.plot(range(0, epochs), losses)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

In [ ]:
# training data (Fv, GT)
train_dataset = (X_train_tensor, y_train)
train_loader = DataLoader(train_dataset, batch_size = 1, shuffle = True)

In [ ]:
model = LinearRegressor()
# summary(model, (1, 9))
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the model
train(model, optimizer, X_train_tensor, y_train_tensor, epochs = 100) 

### KNN and Decision Tree classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3) 
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# Decision tree classifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred_dtree = dtree.predict(X_test)

# Evaluate model performance (assuming regression target)
mse_knn = mean_squared_error(y_test, y_pred_knn)
mse_dtree = mean_squared_error(y_test, y_pred_dtree)

print(f"KNN MSE: {mse_knn:.4f} days")
print(f"Decision Tree MSE: {mse_dtree:.4f} days")